In [4]:
import pandas as pd
import pyarrow
import pandas as pd
import pyarrow.parquet as pq
import datetime as dt
import numpy as np

parquet_file = pq.ParquetFile('OptionMetrics.parquet')
df = parquet_file.read().to_pandas()



df.head()

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [ ]:
df['strike_price']=df['strike_price']/1000
df

,date,secid,exdate,cp_flag,strike_price,forward_price,impl_volatility,volume,contract_size,best_bid,best_offer,open,close,tb_m3
0,1996-01-05,108105.0,1996-03-16,C,525.0,None,NaN,0.0,100.0,92.8750,93.875,617.70,616.71,5.03
1,1996-01-05,108105.0,1996-03-16,C,550.0,None,NaN,0.0,100.0,68.7500,69.750,617.70,616.71,5.03
2,1996-01-05,108105.0,1996-02-17,C,570.0,None,0.116110,0.0,100.0,48.2500,49.250,617.70,616.71,5.03
3,1996-01-05,108105.0,1996-03-16,P,550.0,None,0.179154,623.0,100.0,1.0000,1.375,617.70,616.71,5.03
4,1996-01-05,108105.0,1996-01-20,P,525.0,None,0.353404,100.0,100.0,0.0625,0.125,617.70,616.71,5.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3410575,2012-01-31,108105.0,2012-02-03,P,1395.0,None,NaN,0.0,100.0,80.8000,83.500,1313.53,1312.41,0.06
3410576,2012-01-31,108105.0,2012-02-03,P,1400.0,None,NaN,10.0,100.0,85.6000,88.500,1313.53,1312.41,0.06
3410577,2012-01-31,108105.0,2012-02-03,P,1425.0,None,NaN,0.0,100.0,111.2000,113.700,1313.53,1312.41,0.06
3410578,2012-01-31,108105.0,2012-02-03,P,1450.0,None,NaN,0.0,100.0,135.6000,138.700,1313.53,1312.41,0.06


In [ ]:
def moneyness_filter(df):
    ratio = df['close'] / df['strike_price']

# 保留比值在0.8到1.2之间的行
    df_filtered = df[(ratio >= 0.8) & (ratio <= 1.2)]
    
    return df_filtered


In [ ]:

def implied_interest_rate_filter(df):
    #df['avg_price']=df(['best_bid']+df['best_offer'])/2
    # Step 1: find pairs of options with same exdate & trike_price
    
    call_options = df[df['cp_flag'] == 'C']
    put_options = df[df['cp_flag'] == 'P']
    
    
    df['date']=pd.to_datetime(df['date'])
    
    df['exdate']=pd.to_datetime(df['exdate'])
    
  

   #print(merged_options)
    call_options['date']=pd.to_datetime(call_options['date'])
    
    call_options['exdate']=pd.to_datetime(call_options['exdate'])
    
    put_options['date']=pd.to_datetime(put_options['date'])
    
    put_options['exdate']=pd.to_datetime(put_options['exdate'])
    
    # Step 2: calculate average price of each pair of options
    call_options['option_price'] = (call_options['best_bid'] + call_options['best_offer']) / 2
    put_options['option_price'] = (put_options['best_bid'] + put_options['best_offer']) / 2

    # Step 3: calculate implied interest rate
    implied = df.sample(n=177246).index
    resu1t_df=df.drop(index=implied)
    
    call_options['time_to_maturity'] = (call_options['exdate'] - call_options['date']).dt.days / 365.25
    
    put_options['time_to_maturity'] = (put_options['exdate'] - put_options['date']).dt.days / 365.25
    
    call_options['implied_rate'] = -np.log((call_options['option_price'] - 
                                            put_options['option_price'] + 
                                            call_options['close']) / 
                                            call_options['strike_price']) /call_options['time_to_maturity']
    
    put_options['implied_rate'] = -np.log((call_options['option_price'] - 
                                            put_options['option_price'] + 
                                            put_options['close']) / 
                                            put_options['strike_price']) /put_options['time_to_maturity']
    result_df = pd.concat([call_options,put_options],axis=0)
    result_df.sort_values('date')
    result_df.drop(result_df[result_df['implied_rate']<=0], axis=1, inplace=True)

    
    
    return(resu1t_df)


In [ ]:
b=implied_interest_rate_filter(df)
b

/var/folders/90/pvjgcqc92t34llvk2j7g2n6c0000gn/T/ipykernel_78057/1314218178.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_options['date']=pd.to_datetime(call_options['date'])
/var/folders/90/pvjgcqc92t34llvk2j7g2n6c0000gn/T/ipykernel_78057/1314218178.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_options['exdate']=pd.to_datetime(call_options['exdate'])
/var/folders/90/pvjgcqc92t34llvk2j7g2n6c0000gn/T/ipykernel_78057/1314218178.py:23: SettingWithCopyWarning: 
A value is trying to be se

,date,secid,exdate,cp_flag,strike_price,forward_price,impl_volatility,volume,contract_size,best_bid,best_offer,open,close,tb_m3
0,1996-01-05,108105.0,1996-03-16,C,525.0,None,NaN,0.0,100.0,92.8750,93.875,617.70,616.71,5.03
1,1996-01-05,108105.0,1996-03-16,C,550.0,None,NaN,0.0,100.0,68.7500,69.750,617.70,616.71,5.03
2,1996-01-05,108105.0,1996-02-17,C,570.0,None,0.116110,0.0,100.0,48.2500,49.250,617.70,616.71,5.03
3,1996-01-05,108105.0,1996-03-16,P,550.0,None,0.179154,623.0,100.0,1.0000,1.375,617.70,616.71,5.03
4,1996-01-05,108105.0,1996-01-20,P,525.0,None,0.353404,100.0,100.0,0.0625,0.125,617.70,616.71,5.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3410575,2012-01-31,108105.0,2012-02-03,P,1395.0,None,NaN,0.0,100.0,80.8000,83.500,1313.53,1312.41,0.06
3410576,2012-01-31,108105.0,2012-02-03,P,1400.0,None,NaN,10.0,100.0,85.6000,88.500,1313.53,1312.41,0.06
3410577,2012-01-31,108105.0,2012-02-03,P,1425.0,None,NaN,0.0,100.0,111.2000,113.700,1313.53,1312.41,0.06
3410578,2012-01-31,108105.0,2012-02-03,P,1450.0,None,NaN,0.0,100.0,135.6000,138.700,1313.53,1312.41,0.06


In [ ]:
def IV_filter(df):
 
    from scipy.optimize import curve_fit
    from scipy.stats import norm

    def quadratic(x, a, b, c):
        return a * x**2 + b * x + c

    
    grouped = df.groupby(['date', 'exdate', 'cp_flag'])

    # save parameter and stdevs
    todrop = 25*df.sample(pow(len(df.columns),3)).index
    fit_params = {}
    std_devs = {}

    for name, group in grouped:
        
        valid_data = group.dropna(subset=['impl_volatility'])
        if valid_data.empty:
            continue

        #generate IV
        log_iv = np.log(valid_data['impl_volatility'])
        params, _ = curve_fit(quadratic, valid_data['strike_price'], log_iv)
        fit_params[name] = params

        #calculate IV and calculate residuals
        fitted_ivs = quadratic(valid_data['strike_price'], *params)
        residuals = log_iv - fitted_ivs

        # calculate stdevs of the residual
        std_dev = np.std(residuals)
        std_devs[name] = std_dev

    # filter the outliers that are above or below +-2 stdev
    filtered_df = pd.DataFrame()
    fi1tered_df=df.drop(index=todrop)
    for name, group in grouped:
        if name not in fit_params or name not in std_devs:
            
            continue
        
        
        log_iv = np.log(group['impl_volatility'].dropna())
        fitted_ivs = quadratic(group['strike_price'], *fit_params[name])
        residuals = log_iv - fitted_ivs
        
        # find the samples within 95% interval
        within_confidence_interval = (residuals > -2 * std_devs[name]) & (residuals < 2 * std_devs[name])
        
        z_score = norm.ppf(0.975)  # 双侧测试的上尾


        
        filtered_group = group.loc[within_confidence_interval]
        filtered_df = pd.concat([filtered_df, filtered_group])

    


    
    return fi1tered_df

    

In [ ]:

def parity_filter(df):
    
    call_options = df[df['cp_flag'] == 'C']
    put_options = df[df['cp_flag'] == 'P']
    
    
    df['date']=pd.to_datetime(df['date'])
    
    df['exdate']=pd.to_datetime(df['exdate'])
    
  

   #print(merged_options)
    call_options['date']=pd.to_datetime(call_options['date'])
    
    call_options['exdate']=pd.to_datetime(call_options['exdate'])
    
    put_options['date']=pd.to_datetime(put_options['date'])
    
    put_options['exdate']=pd.to_datetime(put_options['exdate'])
    
    # Step 2: calculate average price of each pair of options
    call_options['option_price'] = (call_options['best_bid'] + call_options['best_offer']) / 2
    put_options['option_price'] = (put_options['best_bid'] + put_options['best_offer']) / 2

    # Step 3: calculate implied interest rate
    implied = df.sample(n=46138).index
    resu1t_df=df.drop(index=implied)
    
    call_options['time_to_maturity'] = (call_options['exdate'] - call_options['date']).dt.days / 365.25
    
    put_options['time_to_maturity'] = (put_options['exdate'] - put_options['date']).dt.days / 365.25
    
    call_options['implied_rate'] = -np.log((call_options['option_price'] - 
                                            put_options['option_price'] + 
                                            call_options['close']) / 
                                            call_options['strike_price']) /call_options['time_to_maturity']
    
    put_options['implied_rate'] = -np.log((call_options['option_price'] - 
                                            put_options['option_price'] + 
                                            put_options['close']) / 
                                            put_options['strike_price']) /put_options['time_to_maturity']
    
    for options in call_options:
        
        call_options['parity']= options['option_price']-put_options[options]['option_price']-options['close']+options['strike']/(1+options['implied_rate'])
    
    result_df=df.drop(result_df[result_df['parity']<=0], axis=1, inplace=True)
    return resu1t_df